In [1]:
import mlflow
import os
from scripts.training_regression_models import Training
from scripts.model_registry import ModelRegistry
from scripts.scoring import Scoring
from scripts.monitoring import Monitoring
from scripts.config_regression import *

## MLflow setup

In [2]:
exp_name = "test1"

if not os.path.exists(local_path_save):
    os.makedirs(local_path_save)

#save all metadata in a sqlite db. Artifacts will be saved on local folder ./mlflow    
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# set up MlFlow axperiment
experiment_id = mlflow.set_experiment(exp_name)

## Model training
Train one or more ml models with fixed or grid-like parameters (define desired parameters in scripts/config.py file. Currently only random forest 'rf' and gradient boosting 'gb' are supported)

All results will be tracked in the MLFlow UI (from a terminal positioned on the root run the command 'mlflow ui --backend-store-uri sqlite:///mlflow.db', then browse to http://127.0.0.1:5000/)

In [3]:
models = models
train = Training(input_data_path_train, local_path_save, year_month_train)
train.preprocess_and_train(models)

/Users/davideposillipo/Documents/Didattica.nosync/DSR_ML_Fundamentals_2023/scripts/preprocessing_regression_models.py:71: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['lpep_pickup_datetime_week'] = df['lpep_pickup_datetime'].dt.week
/Users/davideposillipo/Documents/Didattica.nosync/DSR_ML_Fundamentals_2023/scripts/preprocessing_regression_models.py:71: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['lpep_pickup_datetime_week'] = df['lpep_pickup_datetime'].dt.week
/Users/davideposillipo/Documents/Didattica.nosync/DSR_ML_Fundamentals_2023/scripts/preprocessing_regression_models.py:71: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['lpep_pickup_datetime_week'] = df['lpep_pickup_datetime'].dt.week


###################### training  lr model #########################
rmse_train =  4.220832331011555 
 rmse_test 4.467011358550497 
 mae_train 2.8836415617868596 
 mae_test 3.0855543440720643
###################### training  gb model #########################
  0%|                                                                            | 0/5 [00:00<?, ?trial/s, best loss=?]

/Users/davideposillipo/.pyenv/versions/3.10.9/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


                                                                                                                       
 rmse_train = 
5.728637420057362                                                                                                      
rmse_test                                                                                                              
5.430789491239182                                                                                                      
                                                                                                                       
 rmse_train = 
5.700972672397149                                                                                                      
rmse_test                                                                                                              
5.417670292541586                                                                                                      
          

                                                                                                                       
 rmse_train = 
4.423264582539377                                                                                                      
rmse_test                                                                                                              
6.422131659177675                                                                                                      
[0]	validation_0-rmse:10.73914	validation_1-rmse:10.47410                                                              
[1]	validation_0-rmse:10.15384	validation_1-rmse:9.91189                                                               
[2]	validation_0-rmse:9.61201	validation_1-rmse:9.41444                                                                
[3]	validation_0-rmse:9.11042	validation_1-rmse:8.97390                                                                
[4]	validation_0-rmse:8.6

{'eta': 0.2582958975484436, 'max_depth': 1}

## Model registry management
Identify the best(*) run and store the relative model as 'Production' model. 

Archive former model from 'Production' to 'Archived'

(*) model with lowest test error that doesn't overfit the data: (train -test)/train < of_treshold

In [4]:
#Save best model in model registry as 'Production'
model_reg = ModelRegistry(exp_name, year_month_train, model_name_pref)
model_reg.register_best_run(of_treshold=0.1)

#Archive previous model (from Production to Archived)
model_reg.archive_models(year_month_monitor)

Successfully registered model 'NY2022-01'.
2023/01/04 12:54:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: NY2022-01, version 1


previous models found: deleting them
registering version 1 of model NY2022-01 into Production stage


Created version '1' of model 'NY2022-01'.


## Scoring 
Score latest available data using the Production model identified above, save scored data

In [5]:
scoring = Scoring(year_month_train, model_name_pref, local_path_save)
scoring.preprocess_and_predict(input_data_path_to_score, scored_data_path)

/Users/davideposillipo/Documents/Didattica.nosync/DSR_ML_Fundamentals_2023/scripts/preprocessing_regression_models.py:71: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['lpep_pickup_datetime_week'] = df['lpep_pickup_datetime'].dt.week
/Users/davideposillipo/.pyenv/versions/3.10.9/envs/mlops/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- lpep_pickup_datetime_day
- lpep_pickup_datetime_dayofweek
- lpep_pickup_datetime_hour
- lpep_pickup_datetime_minute
- lpep_pickup_datetime_week
- ...
Feature names seen at fit time, yet now missing:
- trip_distance

  warnings.warn(message, FutureWarning)


ValueError: X has 215 features, but RandomForestRegressor is expecting 208 features as input.

## Monitoring
Evaluate the performance of the previous month model (the archived one), comparing its prediction with the now available observed target

In [ ]:
if year_month_train != '2022-01': #if it is the first passage to production we don't have anything to monitor
    monitor = Monitoring(input_data_path_to_score, scored_data_path, model_name_pref, year_month_monitor, local_path_save)
    monitor.monitor()